# Wave Regression

In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
import os
import os.path as op

import numpy as np
import pandas as pd

import datetime

import matplotlib.pyplot as plt
import pickle as pkl
from scipy.io import loadmat

from scipy import stats
from scipy.optimize import least_squares # used

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
p_db = op.join(os.getcwd(),'..','..','data','Waves')

# database waves (mat file)
p_dat = op.join(p_db, 'cantabria_CSIRO.mat')

# database exploration
data_mat = loadmat(p_dat)

array = data_mat[list(data_mat.keys())[-1]]

In [ ]:
data = pd.DataFrame(
    {
        'hs': data_mat['data'][0][0][5].squeeze(),
        'tp': data_mat['data'][0][0][7].squeeze(),
        'dir': data_mat['data'][0][0][6].squeeze(),
        'year': data_mat['data'][0][0][10].squeeze(),
        'month': data_mat['data'][0][0][11].squeeze(),
        'day': data_mat['data'][0][0][12].squeeze(),
        'hour': data_mat['data'][0][0][13].squeeze(),

    }
)

data['dates'] = pd.to_datetime(data[['year', 'month', 'day', 'hour']])
data = data.drop(['year', 'month', 'day', 'hour'], axis=1).set_index('dates')
data

In [ ]:
# Use pandas library for plotting boxplot
data.boxplot(
    by=data.index.month,
    column = 'hs',
    figsize=(19,10), 
    patch_artist=True,
    boxprops=dict(color='skyblue', facecolor='skyblue'),
    capprops=dict(color='k'),
    whiskerprops=dict(color='k', linewidth=1),
    medianprops=dict(color='w',linewidth=2),
    flierprops=dict(markerfacecolor='royalblue', markersize=6,
                    markeredgecolor='royalblue')
);
plt.show()

## Compute Monthly Means

In [ ]:
#data = data.groupby(pd.Grouper(freq='1D')).mean()
data = data.groupby(pd.Grouper(freq='1M')).mean()
data = data.dropna()

### Create time vector

In [ ]:
#Create a time vector
data['time_vect'] = [day.dayofyear/365.25 + data.index.year[d]-data.index.year.min() \
                       for d, day in enumerate(data.index)]

In [ ]:
def fun_monthly(x, time, hs):    
    return x[0] + x[1]*np.cos(2*np.pi*time) + x[2]*np.sin(2*np.pi*time) - hs

x0 = [1,1,1]

In [ ]:
def fun_monthly_inv(time, a0, a1, a2):    
    return a0 + a1*np.cos(2*np.pi*time) + a2*np.sin(2*np.pi*time)

In [ ]:
def fun_monthly(x, time, hs):    
    return x[0] + x[1]*np.cos(2*np.pi*time) + x[2]*np.sin(2*np.pi*time) + x[3]*np.cos(4*np.pi*time) + x[4]*np.sin(4*np.pi*time) + x[5]*time - hs

x0 = [1,1,1,1,1,1]

In [ ]:
def fun_monthly_inv(time, a0, a1, a2, a3, a4, b0):    

    return a0 + a1*np.cos(2*np.pi*time) + a2*np.sin(2*np.pi*time) + a3*np.cos(4*np.pi*time) + a4*np.sin(4*np.pi*time) + b0*time


In [ ]:
time = data.time_vect.values
hs = data.hs.values

In [ ]:
model_month = least_squares(fun_monthly, x0, args=(time, hs))

In [ ]:
# parameters variance
std = np.sqrt(np.diagonal(np.linalg.inv(model_month.jac.T.dot(model_month.jac))))
covar = np.linalg.inv(model_month.jac.T.dot(model_month.jac))

print('\nOptimal coefficients: ')
print(model_month.x)
print('\nStandar deviation (s.e): ')
print(std)
print('\nCovariance matrix: ')
print(covar)

In [ ]:
hs_mod = fun_monthly_inv(time, *model_month.x)

In [ ]:
plt.figure(figsize = [15, 6])
plt.plot(time, hs, 'k')
plt.plot(time, hs_mod, color = 'crimson')

plt.xlabel('Time')
plt.ylabel('Hs', fontsize = 16)

plt.xlim([0, 30])

In [ ]:
# values standard deviation
size_sim = 1000
data_mc = np.zeros((size_sim, len(data.index)))

for i in range(size_sim):
    data_mc[i,:] = fun_monthly_inv(data.time_vect, *np.random.multivariate_normal(model_month.x, covar))
    
data_mean = np.std(data_mc, axis=0) # should be predictions
data_95 = np.percentile(data_mc, 100-5/2, axis=0) # mean +2 std
data_05 = np.percentile(data_mc, 5/2, axis=0) # mean -2 std
data_std = np.mean(np.std(data_mc, axis=0))
data_std

In [ ]:
fig = make_subplots(rows=3, cols=1, 
                        shared_xaxes=True,
                        specs=[[{'type': 'scatter',
                                 'rowspan': 2}],
                               [None],
                               [{'type': 'scatter'}]])
fig.add_trace(go.Scatter(x=data.index, y=data.hs, mode='markers', 
                             name='Montlhy Mean Temperature (ºC)', marker_color='#636EFA'),
                  row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=hs_mod, mode='lines', 
                             name='LS Reg. model', marker_color='black'),
                  row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=hs_mod+(2*data_std), mode='lines', 
                             name='95% c.i',marker_color='mediumturquoise'),
                  row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=hs_mod-(2*data_std), mode='lines', 
                             name='5% c.i',marker_color='mediumturquoise',
                             fill='tonexty', fillcolor='rgba(0, 181, 204, 0.10)'),
                  row=1, col=1)

fig.update_layout(
        title = 'Real measurements and Least Squares Regression Model comparisons',
        width=1000, height=500,
        xaxis={'type': 'date', 'range': [data.index[0], data.index[-1]],
            'autorange': True}
)
fig.show()